In [15]:
import os
from dotenv import load_dotenv

In [16]:
load_dotenv()

True

In [17]:
from typing import Union, Dict, Set, List, TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
import yfinance as yf
import datetime as dt
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.volume import volume_weighted_average_price
import traceback
import pandas as pd

In [18]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT="Financial_stock_Eval"

In [19]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "Financial_stock_Eval"

In [20]:
@tool
def get_stock_prices(ticker: str) -> Union[Dict, str]:
    """Fetches historical stock price data and technical indicator for a given ticker."""
    try:
        data = yf.download(
            ticker,
            start=dt.datetime.now() - dt.timedelta(weeks=24*3),
            end=dt.datetime.now(),
            interval='1d'
        )
        df= data.copy()
        if len(df.columns[0]) > 1:
            df.columns = [i[0] for i in df.columns]
        data.reset_index(inplace=True)
        data.Date = data.Date.astype(str)
        
        indicators = {}

        # Momentum Indicators
        rsi_series = RSIIndicator(df['Close'], window=14).rsi().iloc[-12:]
        indicators["RSI"] = {date.strftime('%Y-%m-%d'): int(value) for date, value in rsi_series.dropna().to_dict().items()}
        sto_series = StochasticOscillator(
            df['High'], df['Low'], df['Close'], window=14).stoch().iloc[-12:]
        # print(sto_series)
        indicators["Stochastic_Oscillator"] = {date.strftime('%Y-%m-%d'): int(value) for date, value in sto_series.dropna().to_dict().items()}

        macd = MACD(df['Close'])
        macd_series = macd.macd().iloc[-12:]
        # print(macd_series)
        indicators["MACD"] = {date.strftime('%Y-%m-%d'): int(value) for date, value in macd_series.to_dict().items()}
        macd_signal_series = macd.macd_signal().iloc[-12:]
        # print(macd_signal_series)
        indicators["MACD_Signal"] = {date.strftime('%Y-%m-%d'): int(value) for date, value in macd_signal_series.to_dict().items()}
        
        vwap_series = volume_weighted_average_price(
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            volume=df['Volume'],
        ).iloc[-12:]
        indicators["vwap"] = {date.strftime('%Y-%m-%d'): int(value) for date, value in vwap_series.to_dict().items()}
        
        return {'stock_price': data.to_dict(orient='records'), 'indicators': indicators}
    except Exception as e:
        return f"Error fetching price data: {str(e)}"

In [21]:
@tool
def get_financial_metrics(ticker: str) -> Union[Dict, str]:
    """Fetches key financial ratios for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            'pe_ratio': info.get('forwardPE'),
            'price_to_book': info.get('priceToBook'),
            'debt_to_equity': info.get('debtToEquity'),
            'profit_margins': info.get('profitMargins')
        }
    except Exception as e:
        return f"Error fetching ratios: {str(e)}"

In [22]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    stock: str
    
graph_builder = StateGraph(State)

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
llm.invoke("Hello")

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--250f12d0-fa76-405a-babf-06215ccc16b4-0', usage_metadata={'input_tokens': 2, 'output_tokens': 36, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 26}})

In [24]:
FUNDAMENTAL_ANALYST_PROMPT = """
You are a fundamental analyst specializing in evaluating company (whose symbol is {company}) performance based on stock prices, technical indicators, and financial metrics. Your task is to provide a comprehensive summary of the fundamental analysis for a given stock.

You have access to the following tools:
1. **get_stock_prices**: Retrieves the latest stock price, historical price data and technical Indicators like RSI, MACD, Drawdown and VWAP.
2. **get_financial_metrics**: Retrieves key financial metrics, such as revenue, earnings per share (EPS), price-to-earnings ratio (P/E), and debt-to-equity ratio.

### Your Task:
1. **Input Stock Symbol**: Use the provided stock symbol to query the tools and gather the relevant information.
2. **Analyze Data**: Evaluate the results from the tools and identify potential resistance, key trends, strengths, or concerns.
3. **Provide Summary**: Write a concise, well-structured summary that highlights:
    - Recent stock price movements, trends and potential resistance.
    - Key insights from technical indicators (e.g., whether the stock is overbought or oversold).
    - Financial health and performance based on financial metrics.

### Constraints:
- Use only the data provided by the tools.
- Avoid speculative language; focus on observable data and trends.
- If any tool fails to provide data, clearly state that in your summary.

### Output Format:
Respond in the following format:
"stock": "<Stock Symbol>",
"price_analysis": "<Detailed analysis of stock price trends>",
"technical_analysis": "<Detailed time series Analysis from ALL technical indicators>",
"financial_analysis": "<Detailed analysis from financial metrics>",
"final Summary": "<Full Conclusion based on the above analyses>"
"Asked Question Answer": "<Answer based on the details and analysis above>"

Ensure that your response is objective, concise, and actionable."""

In [25]:
tools = [get_stock_prices, get_financial_metrics]
llm_with_tool = llm.bind_tools(tools)

In [26]:
def fundamental_analyst(state: State):
    messages = [
        SystemMessage(content=FUNDAMENTAL_ANALYST_PROMPT.format(company=state['stock'])),
    ]  + state['messages']
    return {
        'messages': llm_with_tool.invoke(messages)
    }

In [27]:
graph_builder.add_node('fundamental_analyst', fundamental_analyst)
graph_builder.add_edge(START, 'fundamental_analyst')
graph_builder.add_node(ToolNode(tools))
graph_builder.add_conditional_edges('fundamental_analyst', tools_condition)
graph_builder.add_edge('tools', 'fundamental_analyst')

In [28]:
graph = graph_builder.compile()

In [29]:
graph = graph_builder.compile()
events = graph.stream({'messages':[('user', 'Should I buy this stock?')],
 'stock': 'TSLA'}, stream_mode='values')
for event in events:
    if 'messages' in event:
        event['messages'][-1].pretty_print()

================================ Human Message =================================

Should I buy this stock?
================================== Ai Message ==================================
Tool Calls:
  get_stock_prices (55b49575-dfba-4be9-a54d-d3c1dbf75213)
 Call ID: 55b49575-dfba-4be9-a54d-d3c1dbf75213
  Args:
    ticker: TSLA
  get_financial_metrics (170b55f1-e48d-48e6-a6cb-94537a36532e)
 Call ID: 170b55f1-e48d-48e6-a6cb-94537a36532e
  Args:
    ticker: TSLA


C:\Users\Abhishek\AppData\Local\Temp\ipykernel_4700\2715306489.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  1 of 1 completed


================================= Tool Message =================================
Name: get_financial_metrics

{"pe_ratio": 94.009254, "price_to_book": 13.137939, "debt_to_equity": 17.407, "profit_margins": 0.063829996}
================================== Ai Message ==================================

"stock": "TSLA",
"price_analysis": "TSLA's stock price has shown considerable volatility over the past year. After reaching a peak around $479 in December 2024, the price experienced a significant decline, hitting lows around $142 in April 2024. In recent months, the price has fluctuated, with a notable drop in early July 2025 followed by a slight rebound. The most recent closing price on 2025-07-10 was $304.59.",
"technical_analysis": "The Relative Strength Index (RSI) for TSLA has consistently remained in the neutral range (40-45) from July 7th to July 10th, 2025, suggesting the stock is neither overbought nor oversold. The Stochastic Oscillator, however, indicated oversold conditions (va

In [30]:
graph = graph_builder.compile()
events = graph.stream({'messages':[('user', 'Should I buy this stock?')],
 'stock': 'AMZN'}, stream_mode='values')
for event in events:
    if 'messages' in event:
        event['messages'][-1].pretty_print()

================================ Human Message =================================

Should I buy this stock?
================================== Ai Message ==================================
Tool Calls:
  get_stock_prices (6106070a-aeb5-42a3-bfa8-c2c1304951db)
 Call ID: 6106070a-aeb5-42a3-bfa8-c2c1304951db
  Args:
    ticker: AMZN
  get_financial_metrics (ca6054bb-9c74-447e-8901-dd5ae6d09d9e)
 Call ID: ca6054bb-9c74-447e-8901-dd5ae6d09d9e
  Args:
    ticker: AMZN


C:\Users\Abhishek\AppData\Local\Temp\ipykernel_4700\2715306489.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  1 of 1 completed


================================= Tool Message =================================
Name: get_financial_metrics

{"pe_ratio": 35.886177, "price_to_book": 7.6578765, "debt_to_equity": 51.641, "profit_margins": 0.1014}
================================== Ai Message ==================================

"stock": "AMZN",
"price_analysis": "Amazon (AMZN) stock has shown an upward trend in recent months. From late February to mid-June 2025, the stock generally trended upwards, reaching a peak of around $223.30 on June 27, 2025. Following this peak, there was a slight pullback, with prices hovering around $219-$223 in early July. More recently, on July 10, 2025, the closing price was $220.69, and on July 11, 2025, it saw a notable drop to $195.05, followed by further declines to $194.49 on July 12 and $192.72 on July 15. The stock experienced a significant drop on August 2, 2024, from $184.07 to $167.89, and continued to decline to $161.02 on August 5, 2024. After this sharp decline, the stock has 

In [31]:
graph = graph_builder.compile()
events = graph.stream({'messages':[('user', 'Should I buy this stock?')],
 'stock': 'BDL'}, stream_mode='values')
for event in events:
    if 'messages' in event:
        event['messages'][-1].pretty_print()

================================ Human Message =================================

Should I buy this stock?
================================== Ai Message ==================================

I cannot provide financial advice or recommendations on whether to buy a stock. However, I can provide a comprehensive fundamental analysis if you provide me with the stock symbol. This analysis will include:

*   **Price Analysis:** Recent stock price movements, trends, and potential resistance levels.
*   **Technical Analysis:** Insights from technical indicators like RSI, MACD, Drawdown, and VWAP.
*   **Financial Analysis:** An evaluation of the company's financial health and performance based on key financial metrics such as revenue, EPS, P/E ratio, and debt-to-equity ratio.

Please provide the stock symbol so I can proceed with the analysis.


In [ ]:
graph = graph_builder.compile()
events = graph.stream({'messages':[('user', 'Should I buy this stock?')],
 'stock': 'TATASTEEL.NS'}, stream_mode='values')
for event in events:
    if 'messages' in event:
        event['messages'][-1].pretty_print()

================================ Human Message =================================

Should I buy this stock?
================================== Ai Message ==================================
Tool Calls:
  get_stock_prices (77f704ca-4177-4663-b3af-bef7f32a4636)
 Call ID: 77f704ca-4177-4663-b3af-bef7f32a4636
  Args:
    ticker: TATASTEEL.NS
  get_financial_metrics (0f3e1446-a82b-4b26-878d-1e8a901b3c70)
 Call ID: 0f3e1446-a82b-4b26-878d-1e8a901b3c70
  Args:
    ticker: TATASTEEL.NS


C:\Users\Abhishek\AppData\Local\Temp\ipykernel_4700\2715306489.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  1 of 1 completed


================================= Tool Message =================================
Name: get_financial_metrics

{"pe_ratio": 9.233334, "price_to_book": 2.1998577, "debt_to_equity": 103.775, "profit_margins": 0.01565}
================================== Ai Message ==================================

"stock": "TATASTEEL.NS",
"price_analysis": "The stock price for TATASTEEL.NS has experienced some volatility in early July 2025. After reaching highs around 165-166 from July 2nd to July 3rd, the price has generally trended downwards, closing at 159.0 on July 9th. On the most recent trading day (July 10th, 2025), the stock saw a slight rebound, closing at 160.66. The current price is slightly below its volume-weighted average price (VWAP) of 161, indicating a minor bearish signal in the very short term.",
"technical_analysis": "As of July 10th, 2025, the technical indicators for TATASTEEL.NS are as follows: The Relative Strength Index (RSI) is 56, and the Stochastic Oscillator is 55. Both indic

In [38]:
graph = graph_builder.compile()
events = graph.stream({'messages':[('user', 'Should I buy this stock?')],
 'stock': 'ADANIENT.NS'}, stream_mode='values')
for event in events:
    if 'messages' in event:
        event['messages'][-1].pretty_print()

================================ Human Message =================================

Should I buy this stock?
================================== Ai Message ==================================
Tool Calls:
  get_stock_prices (306c580c-d379-441a-bb58-44d3a4734942)
 Call ID: 306c580c-d379-441a-bb58-44d3a4734942
  Args:
    ticker: ADANIENT.NS
  get_financial_metrics (6c1dece9-7e56-4896-a729-3e2bf60a821e)
 Call ID: 6c1dece9-7e56-4896-a729-3e2bf60a821e
  Args:
    ticker: ADANIENT.NS


C:\Users\Abhishek\AppData\Local\Temp\ipykernel_4700\2715306489.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  1 of 1 completed


================================= Tool Message =================================
Name: get_financial_metrics

{"pe_ratio": 40.400723, "price_to_book": 6.2464757, "debt_to_equity": 162.597, "profit_margins": 0.072519995}
================================== Ai Message ==================================

"stock": "ADANIENT.NS",
"price_analysis": "The stock has experienced fluctuations, with a recent closing price of 2581.0 as of 2025-07-10. Over the past month, the price initially saw an upward trend, reaching highs around 2646.30 on 2025-06-27, followed by a slight correction and consolidation. The current price is slightly below the Volume Weighted Average Price (VWAP) of 2591, suggesting a minor bearish sentiment in the very short term.",
"technical_analysis": "The Relative Strength Index (RSI) is at 54, indicating a neutral position, neither overbought nor oversold. However, it has decreased from 64 on 2025-06-27, suggesting a cooling off of upward momentum. The Stochastic Oscillator i